# Завдання

Створіть модель згорткової нейронної мережі (CNN) із трьома шарами. Підключіть `Tensorboard`, та відобразіть в ньому зміну розподілу ваг моделі, обчислювальний граф моделі та зміну її метрик.

## Рішення

In [11]:
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=30)
        self.out = nn.Linear(in_features=30, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x,start_dim = 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)

        return x

In [13]:
train_set = torchvision.datasets.FashionMNIST(root="./data",
                                              train = True,
                                              download=True,
                                              transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = 100,
                                           shuffle = True)

In [14]:
tb = SummaryWriter()
model = CNN()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)
tb.add_graph(model, images)
tb.close()

In [15]:
model = CNN()
train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100, shuffle = True)
optimizer = opt.Adam(model.parameters(), lr= 0.01)
criterion = torch.nn.CrossEntropyLoss()

tb = SummaryWriter()

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for images, labels in train_loader:
        images, labels = images, labels
        preds = model(images)

        loss = criterion(preds, labels)
        total_loss+= loss.item()
        total_correct+= get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

    tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
    tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
    tb.add_histogram("conv2.weight", model.conv2.weight, epoch)
    tb.add_histogram("fc1.bias", model.fc1.bias, epoch)
    tb.add_histogram("fc1.weight", model.fc1.weight, epoch)
    tb.add_histogram("fc2.bias", model.fc2.bias, epoch)
    tb.add_histogram("fc2.weight", model.fc2.weight, epoch)
    tb.add_histogram("fc3.bias", model.fc3.bias, epoch)
    tb.add_histogram("fc3.weight", model.fc3.weight, epoch)

    print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)

tb.close()

epoch: 0 total_correct: 44265 loss: 406.95893624424934
epoch: 1 total_correct: 50248 loss: 265.463192358613
epoch: 2 total_correct: 51150 loss: 237.86720491945744
epoch: 3 total_correct: 51652 loss: 226.7817608267069
epoch: 4 total_correct: 51837 loss: 218.85810896754265
epoch: 5 total_correct: 52262 loss: 211.04090775549412
epoch: 6 total_correct: 52435 loss: 206.4378041625023
epoch: 7 total_correct: 52457 loss: 206.2564074844122
epoch: 8 total_correct: 52666 loss: 202.52469559013844
epoch: 9 total_correct: 52638 loss: 202.23483940958977


In [ ]:
%tensorboard --logdir runs

![](01.png)
![](02.png)
![](03.png)
![](04.png)
![](05.png)
![](06.png)
